In [23]:
import argparse
import boto3
import evaluate
import importlib
import json
import lighteval
import os
import pathlib
import requests
import shutil
import sys
import tarfile
import time
import torch
import transformers
import uuid
import wandb

import awswrangler as wr
import numpy as np

from botocore.exceptions import ClientError
from datasets import load_dataset, DatasetDict, Dataset
from datetime import datetime, timezone
from IPython.display import display
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
from sagemaker import image_uris, utils as sm_utils
from sagemaker.huggingface.processing import HuggingFaceProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import FrameworkProcessor
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.session import get_execution_role
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from typing import List, Union, Optional


# Adding ../01_modules or ./01_modules to the system path so that we can load modules from 
# there as well
if '__file__' in globals():
    script_dir = pathlib.Path(__file__).parent.resolve()
else:
    script_dir = pathlib.Path().absolute()
modules_path_in_dev = os.path.abspath(os.path.join(script_dir, '..', '01_modules'))
modules_path_in_prod = os.path.abspath(os.path.join(script_dir, '01_modules'))
if os.path.exists(modules_path_in_dev):
    sys.path.append(modules_path_in_dev)
if os.path.exists(modules_path_in_prod):
    sys.path.append(modules_path_in_prod)


# # Jupyter only reads a local module the first time after 
# # kernel start. Re-running a cell with 
# # "from mymodulename import *" would not change
# # anything, even if the imported module has since changed.
# # As a workaround, we need to directly load the module, 
# # use importlib.reload to reload it and then import * 
import utils
_ = importlib.reload(utils)
import config
_ = importlib.reload(config) 

_ = """
ml.g6.xlarge:   6     250GB
ml.g6.2xlarge:  3
ml.g6.4xlarge:  6
ml.g6.8xlarge:  6
ml.g6.12xlarge: 6
ml.g6.16xlarge: 6
ml.g6.24xlarge: 6
ml.g6.48xlarge: 6

google-bert/bert-base-uncased
distilbert/distilbert-base-uncased
microsoft/deberta-v3-base
FacebookAI/roberta-base
answerdotai/ModernBERT-base
answerdotai/ModernBERT-large
allenai/scibert_scivocab_uncased
google/bigbird-roberta-base
allenai/longformer-base-4096
"""
script_dir

utils.py loaded: v0.2.12
config.py loaded: v0.1


PosixPath('/home/sagemaker-user/sagemaker_research_classification/src/05_tuning_basic')

In [30]:
RUNS = {}
MAX_RUNTIME = 2*24*60*60

In [13]:
MODEL_NAME = 'bert-base-uncased'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bert-subfield-title-s1-0916224638'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bert-topic-title-s1-0917004738'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'bert-subfield-abstract-s1-0917011627'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'bert-topic-abstract-s1-0917012628'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'bert-subfield-fulltext-s1-0917013629'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'bert-topic-fulltext-s1-0917014630'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'bert-subfield-title-s10-0917015631'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'bert-topic-title-s10-0917020632'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bert-subfield-abstract-s10-0917021633'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bert-topic-abstract-s10-0917022634'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': 'bert-subfield-fulltext-s10-0917023635'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': 'bert-topic-fulltext-s10-0917024635'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bert-subfield-title-s100-0917025636'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bert-topic-title-s100-0917030637'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': 'bert-subfield-abstract-s100-0917031638'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': 'bert-topic-abstract-s100-0917032639'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': 'bert-subfield-fulltext-s100-0917033640'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': 'bert-topic-fulltext-s100-0917034641'},
]

In [ ]:
MODEL_NAME = 'bert-base-uncased'
for run in RUNS[MODEL_NAME]:
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(10*60)

Training job created: bert-subfield-abstract-s1-0917011627
Training job created: bert-topic-abstract-s1-0917012628
Training job created: bert-subfield-fulltext-s1-0917013629
Training job created: bert-topic-fulltext-s1-0917014630
Training job created: bert-topic-abstract-s100-0917032639
Training job created: bert-subfield-fulltext-s100-0917033640
Training job created: bert-topic-fulltext-s100-0917034641


In [8]:
MODEL_NAME = 'distilbert-base-uncased'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'distilbert-subfield-title-s1-0917084400'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'distilbert-topic-title-s1-0917093942'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'distilbert-subfield-abstract-s1-0917094943'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'distilbert-topic-abstract-s1-0917095944'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'InProgress: distilbert-subfield-fulltext-s1-0917100945'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'distilbert-topic-fulltext-s1-0917101945'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': False},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': False},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': False},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': False},
]

In [9]:
MODEL_NAME = 'distilbert-base-uncased'
for run in RUNS[MODEL_NAME]:
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(3*60)

Training job created: distilbert-topic-title-s1-0917093942
Training job created: distilbert-subfield-abstract-s1-0917094943
Training job created: distilbert-topic-abstract-s1-0917095944
Training job created: distilbert-subfield-fulltext-s1-0917100945
Training job created: distilbert-topic-fulltext-s1-0917101945
Training job created: distilbert-subfield-title-s10-0917102947
Training job created: distilbert-topic-title-s10-0917103948
Training job created: distilbert-subfield-abstract-s10-0917104949
Training job created: distilbert-topic-abstract-s10-0917105949
Training job created: distilbert-subfield-fulltext-s10-0917110950
Training job created: distilbert-topic-fulltext-s10-0917111951
Training job created: distilbert-subfield-title-s100-0917112952
Training job created: distilbert-topic-title-s100-0917113953
Training job created: distilbert-subfield-abstract-s100-0917114954
Training job created: distilbert-topic-abstract-s100-0917115955
Training job created: distilbert-subfield-fulltext

In [13]:
MODEL_NAME = 'microsoft/deberta-v3-base'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'deberta-subfield-title-s1-0917123950'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': False},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': False},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': False},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': False},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': False},
]

In [14]:
MODEL_NAME = 'microsoft/deberta-v3-base'
for run in RUNS[MODEL_NAME]:
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(3*60)

Training job created: deberta-topic-title-s1-0917131904
Training job created: deberta-subfield-abstract-s1-0917132205
Training job created: deberta-topic-abstract-s1-0917132506
Training job created: deberta-subfield-fulltext-s1-0917132807
Training job created: deberta-topic-fulltext-s1-0917133108
Training job created: deberta-subfield-title-s10-0917133409
Training job created: deberta-topic-title-s10-0917133710
Training job created: deberta-subfield-abstract-s10-0917134010
Training job created: deberta-topic-abstract-s10-0917134312
Training job created: deberta-subfield-fulltext-s10-0917134613
Training job created: deberta-topic-fulltext-s10-0917134914
Training job created: deberta-subfield-title-s100-0917135215
Training job created: deberta-topic-title-s100-0917135516
Training job created: deberta-subfield-abstract-s100-0917135817
Training job created: deberta-topic-abstract-s100-0917140118
Training job created: deberta-subfield-fulltext-s100-0917140419
Training job created: deberta-t

In [31]:
MODEL_NAME = 'roberta-base'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'InProgress: roberta-subfield-title-s1-0917142512'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': False},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': False},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.48xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.48xlarge',
     'done': False},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': False},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.48xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.48xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.48xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.48xlarge',
     'done': False},
]

In [ ]:
MODEL_NAME = 'roberta-base'
for run in reversed(RUNS[MODEL_NAME]):
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(3*60)

Training job created: roberta-topic-fulltext-s100-0917144819


In [19]:
MODEL_NAME = 'answerdotai/ModernBERT-base'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': False},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'not yet'},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': 'not yet'},
]

In [20]:
MODEL_NAME = 'answerdotai/ModernBERT-base'
for run in reversed(RUNS[MODEL_NAME]):
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(3*60)

Training job created: ModernBERT-subfield-title-s1-0917142813


In [24]:
MODEL_NAME = 'allenai/scibert_scivocab_uncased'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': False},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'not yet'},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': 'not yet'},
]

In [25]:
MODEL_NAME = 'allenai/scibert_scivocab_uncased'
for run in reversed(RUNS[MODEL_NAME]):
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(3*60)

Training job created: scibert-subfield-title-s1-0917143225


In [26]:
MODEL_NAME = 'google/bigbird-roberta-base'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': False},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'not yet'},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': 'not yet'},
]

In [27]:
MODEL_NAME = 'google/bigbird-roberta-base'
for run in reversed(RUNS[MODEL_NAME]):
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(3*60)

Training job created: bigbird-subfield-title-s1-0917143526


In [28]:
MODEL_NAME = 'allenai/longformer-base-4096'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': False},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'not yet'},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.12xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.16xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': 'not yet'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.24xlarge',
     'done': 'not yet'},
]

In [29]:
MODEL_NAME = 'allenai/longformer-base-4096'
for run in reversed(RUNS[MODEL_NAME]):
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(3*60)

Training job created: longformer-subfield-title-s1-0917143827
